# 일반 페이지 크롤링하는 경우
- 출처: 구글 여러여러 자료들을 보고 만든 내 예제 코드

In [ ]:
import requests
import os
from bs4 import BeautifulSoup

In [ ]:
response = requests.get("http://commasinem.com/board/gallery/list.html?board_no=8&page=1") # 원하는 주소로 들어가 html? 코드를 가져옴.

In [ ]:
html = response.text

In [ ]:
print(html) # 확인 해보면 크롬에서 f12을 눌렀을 때 나오는 코드들이 그대로 출력됨을 알 수 있음.

In [ ]:
image_urls=[]

In [ ]:
soup = BeautifulSoup(html,'html.parser') # 원하는 부분을 가져오기 위해 모듈을 통해 변환?

In [ ]:
image=soup.find_all("img") # 이 부분은 각 사이트별 상이 할 수 있다. 자세한 것은 사이트 참조.

In [ ]:
print(image)

In [ ]:
for item in image:
    image_urls.append("http:"+item.get("src"))
# 가져온 이미지 주소를 리스트에 저장 / 필요 시 http를 붙여줘야하는 경우도 있음.

In [ ]:
len(image_urls)

In [ ]:
for i in range(40):
    print(image_urls[i])
# range 범위는 바뀔 수 있음.

In [ ]:
result=[]
for i in range(14):
    image_urls=[]
    response = requests.get("http://commasinem.com/board/gallery/list.html?board_no=8&page={}".format(i+1))
    html = response.text
    soup = BeautifulSoup(html,'html.parser')
    image=soup.find_all("img")
    for item in image:
        image_urls.append("http:"+item.get("src"))
    for j in range(3,19):
        result.append(image_urls[j])

In [ ]:
len(result)

In [ ]:
for j in range(224):
    print(result[j])

In [ ]:
# 이미지 주소로 들어가 이미지를 다운 받는 코드
for i in range(224):
    headers = {'User-Agent':'Mozilla/5.0'} # 헤더를 추가하는 이유는 이미지 다운로드는 막는 경우를 방지하기 위함.
    response = requests.get(result[i],headers=headers) # 굳이 추가 해줄 필요는 없지만 이후 막히는 사이트 크롤러에서 다루기는 함.
    image_data = response.content
    filename = os.path.basename(result[i])
    with open(filename, 'wb') as f:
        print('writing to {} ({} bytes)'.format(filename, len(image_data)))
        f.write(image_data)

# 막히는? 사이트를 크롤링 해보자! (구글, 네이버)
- 디도스로 오해하여 차단하거나 이미지가 스크롤을 내릴 때 마다 생성되는 경우, 혹은 그 이외에 사용될 수 있다.
- 위의 코드보다는 시간이 오래걸리지만 사람의 동작과 유사한 액션을 취할 수 있어 막히는 경우가 잘 없다. 

In [ ]:
import requests
from selenium import webdriver # 셀레니엄이라는 웹 어플리케이션 자동테스크 도구를 사용한다.
import time
from bs4 import BeautifulSoup
import re
import math
from selenium.webdriver.common.keys import Keys # 아마 셀레니엄에 특정 값들을 넣어주기 위해 추가한듯.
import os

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36")
driver = webdriver.Chrome('C:/Users/LG/Desktop/chromedriver',chrome_options=options) # 크롬 드라이버 위치 설정
driver.get('about:blank') # 첫 빈 페이지를 연다.
# 각각의 특성이 있는 데 대표적으로 user-agent 경우 위의 크롤러에서 본 것과 같이 헤더를 추가해주는 것이다.
# 어떤 특성이였는지는 까먹었는데 무언가를 지우면 Chromedriver가 켜져 크롬이 열리는 것을 볼 수 있다. 궁금하면 한번 찾아보길......


In [ ]:
driver.get("https://www.google.co.kr/search?q=%ED%8C%A8%EC%85%98%ED%85%8C%EB%9F%AC&rlz=1C1EJFA_enKR778KR778&source=lnms&tbm=isch&sa=X&ved=0ahUKEwibnNrRm6jaAhUBa7wKHSJPCmQQ_AUICigB&biw=1536&bih=743")
time.sleep(1) # 위 주소로 들어간다고 생각하면 된다.(원래는 크롬창에 열려 해당 주소로 들어가지는 것을 눈으로 볼수 있다.)
element = driver.find_element_by_tag_name("body") # time sleep을 주는 이유는 사람의 행동과 유사하게 하기 위함.

# Scroll down
# 구글 이미지에서는 스크롤을 내리면 이미지가 계속 생성되므로 끝까지 내려 이미지 생성을 마치고 코드를 가져오면 된다. 
for i in range(30):
    element.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)

driver.find_element_by_id("smb").click() # 결과 더보기 버튼을 눌러주는 액션을 한다.

for i in range(50):
    element.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)

time.sleep(1)

In [ ]:
html = driver.page_source
driver.close() # 코드를 가져온 후 Chromedriver를 끈다.(창을 닫는다고 생각하면 된다.)

In [ ]:
soup = BeautifulSoup(html,'html.parser')

In [ ]:
image=soup.find_all("img")

In [ ]:
image_urls=[]
for item in image:
    image_urls.append(item.get("src"))

In [ ]:
len(image_urls)

In [ ]:
for i in range(656):
    print(image_urls[i])

In [ ]:
image_urls[22]

In [ ]:
image_urls[645] # 구글 이미지 경우 해본 결과 앞에 대략 20장 정도가 복호화? 되있어서 이미지URL이 되는 것들을 확인 해주어야한다. 

In [ ]:
# 이미지 다운로드 
for i in range(22,646):
    headers = {'User-Agent':'Mozilla/5.0'}
    response = requests.get(image_urls[i],headers=headers)
    image_data = response.content
    filename = os.path.basename("{}_example.jpg".format(i)) # 파일 이름 형식인데 여기서는 jpg형식임을 따로 해줘야 하더라.
    with open(filename, 'wb') as f:
        print('writing to {} ({} bytes)'.format(filename, len(image_data)))
        f.write(image_data)

# PS.
- 사이트마다 경우가 다 상이하므로 각각의 경우마다 코드를 수정해줘야한다.(일부 같은 경우도 있긴 하더라.)
- 이 코드를 그냥 실행하기 보다는 새 창을 띄워 해보는 것이 좋다.(위의 이유 때문)
- 다른 IDE를 써도 무방하긴 하겠지만 각각의 오류로 심신의 피로를 느낄 수 있을 것이다. 고로 Jupyter notebook을 사용하는 것을 강력히 추천한다.
- 위의 코드 안될 수도 있음! (왜냐면 내가 테스트를 안해봤기 때문) 그래도 아마 될꺼야.....예전에 됬던걸 가져와서....